In [1]:
# blanket import modules
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import pprint
from scipy.stats import linregress
import requests
import gmaps
import gmaps.geojson_geometries
from api_key import gkey
from matplotlib.colors import to_hex
from matplotlib.cm import inferno
#api key
gmaps.configure(api_key=gkey)

In [2]:
happy_cs = "Output/happy_comb-data.csv"

In [15]:
happy_df = pd.read_csv(happy_cs)
#trying to use this at the end 
happy_country=happy_df['Country']
happy_df

,RANK,Country,Happiness score,Explained by: Healthy life expectancy,Sunshine hours per Year,Average Income,Alcohol Consumption (L/year)
0,1,Finland,7.821,0.775,1858.00,18742.0,10.65
1,2,Denmark,7.636,0.777,1932.00,20304.0,10.13
2,3,Iceland,7.557,0.803,1326.00,19300.0,9.21
3,4,Switzerland,7.512,0.822,1932.75,25787.0,11.23
4,5,Netherlands,7.415,0.787,1662.00,19690.0,9.67
...,...,...,...,...,...,...,...
93,136,India,3.777,0.471,2517.02,1314.0,5.61
94,137,Zambia,3.760,0.306,2965.47,1045.0,4.46
95,138,Malawi,3.750,0.388,2795.87,733.0,4.08
96,139,Tanzania,3.702,0.425,3032.85,992.0,12.04


In [5]:
#making initial map with lines around country to fill later
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')
fig = gmaps.figure()
happy_layer=gmaps.geojson_layer(countries_geojson)
fig.add_layer(happy_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# need to get the min and the max of happiness
# happy_df=happy_df.set_index(["Country"])

In [7]:
happy_df.dtypes

RANK                                       int64
Country                                   object
Happiness score                          float64
Explained by: Healthy life expectancy    float64
Sunshine hours per Year                  float64
Average Income                           float64
Alcohol Consumption (L/year)             float64
dtype: object

In [8]:
happy_col=happy_df["Happiness score"]
h_range=[]

for x in happy_col:
    happ_conv= x / 10
    h_range.append(happ_conv)
    
# happy_range=happ_max-happ_min

happy_df['Happiness Range'] = h_range
happy_df

,RANK,Country,Happiness score,Explained by: Healthy life expectancy,Sunshine hours per Year,Average Income,Alcohol Consumption (L/year),Happiness Range
0,1,Finland,7.821,0.775,1858.00,18742.0,10.65,0.7821
1,2,Denmark,7.636,0.777,1932.00,20304.0,10.13,0.7636
2,3,Iceland,7.557,0.803,1326.00,19300.0,9.21,0.7557
3,4,Switzerland,7.512,0.822,1932.75,25787.0,11.23,0.7512
4,5,Netherlands,7.415,0.787,1662.00,19690.0,9.67,0.7415
...,...,...,...,...,...,...,...,...
93,136,India,3.777,0.471,2517.02,1314.0,5.61,0.3777
94,137,Zambia,3.760,0.306,2965.47,1045.0,4.46,0.3760
95,138,Malawi,3.750,0.388,2795.87,733.0,4.08,0.3750
96,139,Tanzania,3.702,0.425,3032.85,992.0,12.04,0.3702


In [9]:
# We will need to scale the Happiness values to lie between 0 and 1, this sets up for color ranges below
happ_max1=happy_df["Happiness Range"].max()
happ_min1=happy_df["Happiness Range"].min()
hap_range=happ_max1-happ_min1

def calculate_color(h_range):
    
#     Convert the GINI coefficient to a color

    # make gini a number between 0 and 1
    normalized_range = (h_range - happ_min1) / hap_range

    # invert gini so that high inequality gives dark color
    inverse_hap = 1.0 - normalized_range

    # transform the gini coefficient to a matplotlib color
    mpl_color = inferno(inverse_hap)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [16]:
colors = []
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    try:
        happy_df = happy_df['Country']
        color = calculate_color(h_range)
    except KeyError:
        # no happyy score for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [14]:

fig = gmaps.figure()
happy_layer2 = gmaps.geojson_layer(
    happy_country,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(happy_layer2)
fig

TypeError: Object of type Series is not JSON serializable